<a href="https://colab.research.google.com/github/MikhailBolt/school_DS/blob/main/%D0%A8%D0%BA%D0%BE%D0%BB%D0%B0_DS_%D0%98%D1%82%D0%BE%D0%B3%D0%BE%D0%B2%D1%8B%D0%B9_%D0%BF%D1%80%D0%BE%D0%B5%D0%BA%D1%82%5B%D0%91%D0%BE%D0%BB%D1%82%D1%8B%D1%88%D0%B5%D0%B2_%D0%9C%D0%B8%D1%85%D0%B0%D0%B8%D0%BB%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Дейтинг-индустрия
https://cloud.mail.ru/public/yvsP/gcRYW5qC4
1. В ноутбуке “sample reader edges .ipynb” - описание колонок и задача
2. Дополнительно:
a. Необходимо сформировать 2 колоды пользователей для каждого:
i. Те, которых пользователь с максимальной вероятностью лайкнет
ii. Те, с кем максимальной вероятностью будет мэтч (взаимный
лайк)
b. Упаковать в сервис по API (flask)

При взаимном лайке возникает возможность общения

Для упрощения можно допустить: если rating >=6, то это лайк, иначе - дизлайк

Необходимо написать код, который поможет найти для следуюших пользователей, которые они с максимальной вероятностью залайкают (за вычетом известных в датасете пар). Для каждого id - не более 10 пользователей

id: 98447, 111944, 111930, 82091, 125299, 125298, 17335, 60356

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import pearsonr

In [ ]:
#Заргужаем даныне

# В датасете данные о пользователе
# 1ая колонка id
# 2ая колонка gender: "M" - male, "F" - female,  "U" - unknown
# по остальным колонкам недоступна информация
#df_info = pd.read_csv('info.csv')
# В датасете данные, как пользователь с id_from оценил (rating) пользователя id_to
#df_ratings = pd.read_csv("rec-libimseti-dir.edges", delim_whitespace=True, skiprows=1, names=["id_from", "id_to", "rating"])

# Загрузка данных
# В датасете данные, как пользователь с id_from оценил (rating) пользователя id_to

users  = pd.read_csv('info.csv')
ratings  = pd.read_csv("rec-libimseti-dir.edges", delim_whitespace=True, skiprows=1, names=["id_from", "id_to", "rating"])

<ipython-input-2-8834fe1b7076>:14: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  ratings  = pd.read_csv("rec-libimseti-dir.edges", delim_whitespace=True, skiprows=1, names=["id_from", "id_to", "rating"])


In [ ]:
users.columns = users.columns.str.strip()

In [ ]:
# Посмотрим на первые несколько строк каждого файла для анализа структуры данных
users.head(), users.columns

(   id gender  1   2   3   4   5   6   7   8   9  10  11        12
 0   1      F  1  45  90  15   8  13  63  57  96  42  87  0.934487
 1   2      F  1  46  81   6   9  31  22  79  82  75  53  0.675761
 2   3      U  4  69  64   2  18  44  54  80  50  89  38  0.927339
 3   4      F  3  46  90   9  42  54  59  86  60  65  25  0.176650
 4   5      F  3  79  83   1  45  57   1  58  34  23  33  0.553003,
 Index(['id', 'gender', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11',
        '12'],
       dtype='object'))

In [ ]:
ratings.head()

,id_from,id_to,rating
0,1,133,8
1,1,720,6
2,1,971,10
3,1,1095,7
4,1,1616,10


In [ ]:
# Преобразуем таблицу с рейтингами в формат для удобного анализа

likes = ratings[ratings['rating'] >= 6]
dislikes = ratings[ratings['rating'] < 6]

In [ ]:
# Функция для вычисления похожих пользователей по коэффициенту Пирсона
def get_similar_users(user_id, target_ids, user_data, n=10):
    similarities = []
    target_user_data = user_data[user_data['id'] == user_id].drop(['id', 'gender'], axis=1).values.flatten()

    for target_id in target_ids:
        if target_id == user_id:
            continue  # Пропускаем самого пользователя
        target_data = user_data[user_data['id'] == target_id].drop(['id', 'gender'], axis=1).values.flatten()
        # Коэффициент корреляции Пирсона
        correlation, _ = pearsonr(target_user_data, target_data)
        similarities.append((target_id, correlation))

    # Сортировка по убыванию коэффициента корреляции
    sorted_similarities = sorted(similarities, key=lambda x: x[1], reverse=True)
    return sorted_similarities[:n]

In [ ]:
# Пример получения похожих пользователей для одного из целевых ID
target_user_id = 98447
target_ids = users['id'].tolist()
recommendations = get_similar_users(target_user_id, target_ids, users)
recommendations

[(173060, 0.9847972974638955),
 (167562, 0.9798095166352936),
 (163170, 0.9768664822482619),
 (92303, 0.9748919392843226),
 (64228, 0.9732145551966102),
 (135805, 0.971593799410988),
 (84517, 0.9709362055217698),
 (165608, 0.9706385031728323),
 (183885, 0.9703469203915995),
 (193429, 0.9692144568770185)]

In [ ]:
def recommend_likes_and_matches(target_ids, likes_df, user_df, n=10):
    recommendations = {}

    for user_id in target_ids:
        # 1. Найдем похожих пользователей по коэффициенту Пирсона
        similar_users = get_similar_users(user_id, user_df['id'].tolist(), user_df, n)

        # 2. Колода для вероятного лайка
        like_deck = [u[0] for u in similar_users if u[1] > 0]  # выбираем пользователей с положительной корреляцией

        # 3. Колода для мэтчей (взаимных лайков)
        match_deck = []
        user_likes = set(likes_df[likes_df['id_from'] == user_id]['id_to'])

        for u, similarity in similar_users:
            target_likes = set(likes_df[likes_df['id_from'] == u]['id_to'])
            if user_id in target_likes and u in user_likes:
                match_deck.append(u)

        # Оставляем максимум по 10 рекомендаций
        recommendations[user_id] = {
            "likely_to_like": like_deck[:n],
            "likely_to_match": match_deck[:n]
        }

    return recommendations

# Целевые пользователи
target_user_ids = [98447, 111944, 111930, 82091, 125299, 125298, 17335, 60356]

# Получение рекомендаций
recommendations = recommend_likes_and_matches(target_user_ids, likes, users)
print(recommendations)

{98447: {'likely_to_like': [173060, 167562, 163170, 92303, 64228, 135805, 84517, 165608, 183885, 193429], 'likely_to_match': []}, 111944: {'likely_to_like': [206580, 82645, 147378, 79617, 194211, 19160, 69727, 105177, 153346, 204267], 'likely_to_match': []}, 111930: {'likely_to_like': [164190, 142033, 30095, 29861, 203727, 129816, 171691, 77253, 150333, 61251], 'likely_to_match': []}, 82091: {'likely_to_like': [35180, 170714, 188572, 10706, 7388, 212990, 12270, 24846, 96671, 52993], 'likely_to_match': []}, 125299: {'likely_to_like': [210285, 90728, 204654, 150417, 31937, 141975, 72946, 86165, 111810, 14994], 'likely_to_match': []}, 125298: {'likely_to_like': [78141, 193950, 105569, 199178, 168340, 167912, 171965, 132788, 86250, 699], 'likely_to_match': []}, 17335: {'likely_to_like': [39721, 91897, 53562, 70277, 215389, 6978, 74060, 18166, 72171, 210318], 'likely_to_match': []}, 60356: {'likely_to_like': [220492, 70895, 140146, 82327, 151988, 190641, 28718, 205879, 34554, 49958], 'likel